# I. US County Race Population

In [1]:

import pandas as pd
import numpy as np 
from datetime import datetime  


def diagnose_Rate(T, **kwargs):
    info_list = []
    for col in T.columns:
        if col in ['Rate_Black', 'Rate_White', 'Rate_Parity']:
            d = {'name': col}
            for k, v in kwargs.items():
                d[k] = v
                
            for k,v in T[col].describe().to_dict().items():
                d[k] = v
            info_list.append(d)

    Report = pd.DataFrame(info_list)
    return Report


# US_COUNTY_POPULATION[US_COUNTY_POPULATION['Total_Whole'] > 100000]

path = 'Data/US_COUNTY_POPULATION.csv'
US_COUNTY_POPULATION = pd.read_csv(path)
US_COUNTY_POPULATION.head()

,GEOID,State,County,Geographic Area Name,Total_Whole,Total_White,Total_Black,Total_AIndA,Total_Asian,Total_WhiteC,Total_BlackC,Total_AIndAC,Total_AsianC
0,1001,Alabama,Autauga,"Autauga, Alabama",55869,42607,11237,266,656,43563,11784,575,950
1,1003,Alabama,Baldwin,"Baldwin, Alabama",223234,195198,19593,1742,2380,199103,21354,3603,3351
2,1005,Alabama,Barbour,"Barbour, Alabama",24686,12134,11906,170,116,12380,12117,297,152
3,1007,Alabama,Bibb,"Bibb, Alabama",22394,17191,4763,103,48,17420,4907,207,97
4,1009,Alabama,Blount,"Blount, Alabama",57826,55412,978,370,185,56167,1276,820,312


In [2]:
# StateNum = US_COUNTY_POPULATION.groupby('State').sum()[ 'Total_Whole']
# # for state, row in StateNum.iterrows():
# #     print(state, row)


# states = ['Texas', 'Virginia', 'Illinois', 'Tennessee', 'Ohio', 'Indiana',
#        'Wisconsin', 'New York', 'Pennsylvania', 'California', 'West Virginia',
#        'South Carolina', 'North Carolina', 'New Jersey', 'Oregon', 'Maine']
# sum([v for k, v in StateNum.to_dict().items() if k in states])
    

In [3]:

STATE_VACCINE = {}


In [4]:
print("Enter the date of collecting data (Today's data in YYYY-MM-DD):")
Today_Date = input()

assert Today_Date == str(datetime.now().date())

print('Hello, ' + Today_Date)


Enter the date of collecting data (Today's data in YYYY-MM-DD):
2021-04-17
Hello, 2021-04-17


# II. API States

## [!] A. California* (Done)


https://covid19.ca.gov/vaccines/#California-vaccines-dashboard


https://data.ca.gov/dataset/covid-19-vaccine-progress-dashboard-data 


https://data.ca.gov/dataset/covid-19-vaccine-progress-dashboard-data/resource/c020ef6b-2116-4775-b11d-9df2875096ab # click here



Statewide COVID-19 Vaccines Administered By CountyCSV


Direct Vaccine Rate 

No Fully Rate information

In [7]:
# path = 'TMP/April08/California/covid19vaccinesbycounty.csv'

'TMP/April09/California/covid19vaccinesbycounty.csv'
'TMP/April10/California/covid19vaccinesbycounty.csv'

'TMP/April12/California/c020ef6b-2116-4775-b11d-9df2875096ab.csv'

'TMP/April14/California/covid19vaccinesbycounty.csv'


'TMP/April16/California/covid19vaccinesbycounty.csv'

'TMP/April16/California/covid19vaccinesbycounty.csv'

In [8]:


Date = Today_Date
import os

while True:
    Path = input('Enter California Manually collected Data:  ')
    if os.path.isfile(Path):
        print('File Exists!')
        x = input('Enter "Yes" if the file contains the information of the date you wanted:  ')
        if x =='Yes':
            break
    else:
        print('Wrong file path...')
        
        
print('\nThe Path we have:')
print(Path)

from datetime import datetime 
import numpy as np

import requests
import time

    

# CalInfo: county total vaccine number each date
CalInfo = pd.read_csv(Path)
CalInfo = CalInfo[CalInfo['administered_date'] == Date].reset_index(drop = True)



while len(CalInfo) == 0:
    print('There is no information for current Date:', Date)
    Date = input('Please Enter a new One:')
    CalInfo = pd.read_csv(Path)
    CalInfo = CalInfo[CalInfo['administered_date'] == Date].reset_index(drop = True)
    

print('We get California Information for Date:', Date)
CalInfo = CalInfo[['county', 'california_flag', 'cumulative_at_least_one_dose']]
CalInfo['Geographic Area Name'] = CalInfo['county'] + ', ' + CalInfo['california_flag']
CalInfo

Enter California Manually collected Data:  TMP/April16/California/covid19vaccinesbycounty.csv
File Exists!
Enter "Yes" if the file contains the information of the date you wanted:  Yes

The Path we have:
TMP/April16/California/covid19vaccinesbycounty.csv
There is no information for current Date: 2021-04-17
Please Enter a new One:2021-04-16
There is no information for current Date: 2021-04-16
Please Enter a new One:2021-04-15
We get California Information for Date: 2021-04-15


,county,california_flag,cumulative_at_least_one_dose,Geographic Area Name
0,All CA Counties,NaN,15877751,NaN
1,All CA and Non-CA Counties,NaN,16176869,NaN
2,Outside California,Not in California,296384,"Outside California, Not in California"
3,Unknown,Not in California,2734,"Unknown, Not in California"
4,Alameda,California,784346,"Alameda, California"
...,...,...,...,...
57,Tulare,California,137617,"Tulare, California"
58,Tuolumne,California,20652,"Tuolumne, California"
59,Ventura,California,348127,"Ventura, California"
60,Yolo,California,93716,"Yolo, California"


In [9]:

def get_California_Raw_and_postprocess(US_COUNTY_POPULATION):
    

    state = 'California'
    ca_county_list = [i.lower().replace(' ','') for i in US_COUNTY_POPULATION[US_COUNTY_POPULATION['State'] == 'California']['County'].values]

    L = []
    error_list = []
    for i in ca_county_list:

        api_url = 'https://files.covid19.ca.gov/data/vaccine-equity/race-ethnicity/vaccines_by_race_ethnicity_{}.json'.format(i)

        resp = requests.get(api_url)# .to_json()
        data = resp.json() # resp

        try:

            d = {}
            d['County'] = data['meta']['REGION']
            d['Latest_Date'] = data['meta']['LATEST_ADMIN_DATE']
            for i in data['data']:
                d[i['CATEGORY']] = i['METRIC_VALUE']
            L.append(d)

        except:
            print(i)
            error_list.append(i)

    CA_County_Vaccine_Race = pd.DataFrame(L)
    
    return CA_County_Vaccine_Race





def standardize_California(CA_County_Vaccine_Race, US_COUNTY_POPULATION):
    
    


    # IL_County_Vaccine_Race = pd.DataFrame(L)
    state = 'California'

    new_L = []


    for idx, row in CA_County_Vaccine_Race.iterrows():
        d = row.to_dict()

        new_d = {}


        new_d['State'] = state
        new_d['County'] = d['County']
        
        RenameCounties = {
            # 'Mckean': 'McKean'
        }
        if new_d['County'] in RenameCounties:
            new_d['County'] = RenameCounties[new_d['County']]

        new_d['Geographic Area Name'] = new_d['County'] + ', ' + new_d['State']



        # new_d['Numb_White'] = np.nan
        new_d['Rate_White_V'] = d['White']

        # new_d['Numb_Black'] = np.nan
        new_d['Rate_Black_V'] = d['Black']

        # new_d['Numb_Asian'] = np.nan
        new_d['Rate_Asian_V'] = d['Asian American']

        # new_d['Numb_AIndA'] = np.nan
        new_d['Rate_AIndA_V'] = d['American Indian or Alaska Native (AI/AN)']

        # new_d['Numb_Other'] = np.nan
        new_d['Rate_Other_V'] = d['Other']

        new_d['Date'] = d['Latest_Date']

        new_L.append(new_d)


    Data = pd.DataFrame(new_L)

    not_included = list(set(Data['Geographic Area Name'].unique()) - set(US_COUNTY_POPULATION['Geographic Area Name'].unique()))
    if len(not_included) > 0:
        print('Not included Counties', not_included)

    
    T = pd.merge(Data, US_COUNTY_POPULATION[US_COUNTY_POPULATION.columns[3:]], on = 'Geographic Area Name')

        
        
    T = T[list(T.columns[:3]) + list(T.columns[3:].sort_values())]
    
    return T
    

    

CA_County_Vaccine_Race = get_California_Raw_and_postprocess(US_COUNTY_POPULATION)

# CA_County_Vaccine_Race.to_csv('FinalResult/Post_California.csv')
# print(CA_County_Vaccine_Race.shape)
CA_County_Vaccine_Race.head()

,County,Latest_Date,American Indian or Alaska Native (AI/AN),Asian American,Black,Latino,Multi-race,Native Hawaiian or Other Pacific Islander (NHPI),White,Other,Unknown
0,Alameda,2021-04-15,0.002477,0.261814,0.062003,0.117653,0.119357,0.006131,0.277664,0.095123,0.057777
1,Alpine,2021-04-15,0.017094,0.008547,0.002442,0.010989,0.003663,0.000000,0.178266,0.008547,0.770452
2,Amador,2021-04-15,0.008219,0.016124,0.004956,0.078424,0.039777,0.003325,0.704812,0.088525,0.055838
3,Butte,2021-04-15,0.011041,0.029787,0.007939,0.100773,0.171608,0.001241,0.579071,0.064288,0.034253
4,Calaveras,2021-04-15,0.004705,0.014711,0.004765,0.060393,0.018642,0.002085,0.587076,0.083979,0.223645


In [10]:
T = standardize_California(CA_County_Vaccine_Race, US_COUNTY_POPULATION)
state = 'California'

# T.to_csv('FinalResult/T_' + state +'.csv', index = False)
print(T.shape)
T.head()

(58, 18)


,State,County,Geographic Area Name,Date,Rate_AIndA_V,Rate_Asian_V,Rate_Black_V,Rate_Other_V,Rate_White_V,Total_AIndA,Total_AIndAC,Total_Asian,Total_AsianC,Total_Black,Total_BlackC,Total_White,Total_WhiteC,Total_Whole
0,California,Alameda,"Alameda, California",2021-04-15,0.002477,0.261814,0.062003,0.095123,0.277664,17797,37797,540391,594001,184368,214392,823593,899377,1671329
1,California,Alpine,"Alpine, California",2021-04-15,0.017094,0.008547,0.002442,0.008547,0.178266,290,310,18,27,4,26,767,813,1129
2,California,Amador,"Amador, California",2021-04-15,0.008219,0.016124,0.004956,0.088525,0.704812,925,1644,665,1043,1066,1374,35636,36844,39752
3,California,Butte,"Butte, California",2021-04-15,0.011041,0.029787,0.007939,0.064288,0.579071,5547,9925,10984,14403,4158,7087,187742,197213,219186
4,California,Calaveras,"Calaveras, California",2021-04-15,0.004705,0.014711,0.004765,0.083979,0.587076,905,1858,859,1399,498,836,41752,43344,45905


In [11]:
not_included = list(set(T['Geographic Area Name'].unique()) - set(CalInfo['Geographic Area Name'].unique()))
if len(not_included) > 0:
    print('Not included Counties', not_included)

    
not_included = list(set(CalInfo['Geographic Area Name'].unique()) - set(T['Geographic Area Name'].unique()))
if len(not_included) > 0:
    print('Not included Counties', not_included)


Not included Counties [nan, 'Outside California, Not in California', 'Unknown, Not in California']


In [12]:
T2 = pd.merge(T, CalInfo[['Geographic Area Name', 'cumulative_at_least_one_dose']] , on = 'Geographic Area Name')
print(T2.shape)
T2.head()

(58, 19)


,State,County,Geographic Area Name,Date,Rate_AIndA_V,Rate_Asian_V,Rate_Black_V,Rate_Other_V,Rate_White_V,Total_AIndA,Total_AIndAC,Total_Asian,Total_AsianC,Total_Black,Total_BlackC,Total_White,Total_WhiteC,Total_Whole,cumulative_at_least_one_dose
0,California,Alameda,"Alameda, California",2021-04-15,0.002477,0.261814,0.062003,0.095123,0.277664,17797,37797,540391,594001,184368,214392,823593,899377,1671329,784346
1,California,Alpine,"Alpine, California",2021-04-15,0.017094,0.008547,0.002442,0.008547,0.178266,290,310,18,27,4,26,767,813,1129,819
2,California,Amador,"Amador, California",2021-04-15,0.008219,0.016124,0.004956,0.088525,0.704812,925,1644,665,1043,1066,1374,35636,36844,39752,15939
3,California,Butte,"Butte, California",2021-04-15,0.011041,0.029787,0.007939,0.064288,0.579071,5547,9925,10984,14403,4158,7087,187742,197213,219186,72549
4,California,Calaveras,"Calaveras, California",2021-04-15,0.004705,0.014711,0.004765,0.083979,0.587076,905,1858,859,1399,498,836,41752,43344,45905,16790


In [13]:
T2.columns

Index(['State', 'County', 'Geographic Area Name', 'Date', 'Rate_AIndA_V',
       'Rate_Asian_V', 'Rate_Black_V', 'Rate_Other_V', 'Rate_White_V',
       'Total_AIndA', 'Total_AIndAC', 'Total_Asian', 'Total_AsianC',
       'Total_Black', 'Total_BlackC', 'Total_White', 'Total_WhiteC',
       'Total_Whole', 'cumulative_at_least_one_dose'],
      dtype='object')

In [14]:
for Race in ['White', 'Black']:
    T2['Numb_' + Race]        = T2['cumulative_at_least_one_dose'] * T2['Rate_' + Race + '_V']
    T2['Rate_' + Race]       = T2['Numb_' + Race] / T2['Total_' + Race]
    # T2['Rate_' + Race + 'C'] = T2['Num_' + Race] / T2['Total_' + Race + 'C']

    
cols = ['State', 'County', 'Geographic Area Name', 'Date', 'Numb_White',
       'Rate_White', 'Numb_Black', 'Rate_Black',
       'Total_AIndA', 'Total_AIndAC', 'Total_Asian', 'Total_AsianC',
       'Total_Black', 'Total_BlackC', 'Total_White', 'Total_WhiteC',
       'Total_Whole', 'cumulative_at_least_one_dose', ]
T = T2[cols]
T.head()

,State,County,Geographic Area Name,Date,Numb_White,Rate_White,Numb_Black,Rate_Black,Total_AIndA,Total_AIndAC,Total_Asian,Total_AsianC,Total_Black,Total_BlackC,Total_White,Total_WhiteC,Total_Whole,cumulative_at_least_one_dose
0,California,Alameda,"Alameda, California",2021-04-15,217784.647744,0.264432,48631.805038,0.263776,17797,37797,540391,594001,184368,214392,823593,899377,1671329,784346
1,California,Alpine,"Alpine, California",2021-04-15,145.999854,0.190352,1.999998,0.500000,290,310,18,27,4,26,767,813,1129,819
2,California,Amador,"Amador, California",2021-04-15,11233.998468,0.315243,78.993684,0.074103,925,1644,665,1043,1066,1374,35636,36844,39752,15939
3,California,Butte,"Butte, California",2021-04-15,42011.021979,0.223770,575.966511,0.138520,5547,9925,10984,14403,4158,7087,187742,197213,219186,72549
4,California,Calaveras,"Calaveras, California",2021-04-15,9857.006040,0.236085,80.004350,0.160651,905,1858,859,1399,498,836,41752,43344,45905,16790


In [15]:
T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']
# T['Rate_ParityC'] = T['Rate_WhiteC'] - T['Rate_BlackC']
STATE_VACCINE[state] = T
print([i for i in STATE_VACCINE])
diagnose_Rate(T)

['California']


<ipython-input-15-266ad6d67fc6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']


,name,count,mean,std,min,25%,50%,75%,max
0,Rate_White,58.0,0.205603,0.081856,0.035407,0.131501,0.210392,0.263380,0.410305
1,Rate_Black,58.0,0.156300,0.081168,0.018684,0.112382,0.145948,0.210693,0.500000
2,Rate_Parity,58.0,0.049304,0.097007,-0.309648,-0.005867,0.024463,0.101755,0.304135


# III. Table States

## [!] B. Tennessee* (Done)


https://www.tn.gov/health/cedep/ncov/data/downloadable-datasets.html

Covid Vaccine County Demographics


95 counties





In [16]:
import pandas as pd


url = 'https://www.tn.gov/content/dam/tn/health/documents/cedep/novel-coronavirus/datasets/COVID_VACCINE_COUNTY_DEMOGRAPHICS.XLSX'
RawData = pd.read_excel(url)
RawData.tail()  # RawData

,DATE,COUNTY,CATEGORY,CAT_DETAIL,RECIPIENT_COUNT
88862,2021-04-16,WILSON,ETHN,UNKNOWN,5796.0
88863,2021-04-16,WILSON,SEX,F,24935.0
88864,2021-04-16,WILSON,SEX,M,18975.0
88865,2021-04-16,WILSON,SEX,O,19.0
88866,2021-04-16,WILSON,SEX,U,75.0


In [17]:
######################################################
Date = Today_Date 
######################################################  


# RawData2 = RawData

while (RawData['DATE'] == Date).sum() == 0:
    print('There is no information for current Date:', Date)
    Date = input('Please Enter a new One:')

    
    

print('We get Tennessee Information for Date:', Date)
RawData = RawData[RawData['DATE'] == Date]

RawData.head()

There is no information for current Date: 2021-04-17
Please Enter a new One:2021-04-16
We get Tennessee Information for Date: 2021-04-16


,DATE,COUNTY,CATEGORY,CAT_DETAIL,RECIPIENT_COUNT
87739,2021-04-16,ANDERSON,RACE,ASIAN,281.0
87740,2021-04-16,ANDERSON,RACE,BLACK OR AFRICAN AMERICAN,543.0
87741,2021-04-16,ANDERSON,RACE,OTHER/MULTIRACIAL,4614.0
87742,2021-04-16,ANDERSON,RACE,UNKNOWN,2610.0
87743,2021-04-16,ANDERSON,RACE,WHITE,19904.0


In [18]:


RawData = RawData[RawData['CATEGORY'].isin(['RACE', 'ETHN'])]


L= []

for county, record in RawData.groupby('COUNTY'):
    # print(county)
    D = {}
    D['County'] = county

    for idx, row in record.iterrows():
        d = row.to_dict()
        # print(d)
        
        item = d['CAT_DETAIL']
        if item == 'UNKNOWN': 
            item = d['CATEGORY'] + '-' + item
        
        D[item] = d['RECIPIENT_COUNT']
    # print(D)
    L.append(D)

Tennessee_County_Vaccine_Race = pd.DataFrame(L)


In [19]:
# Tennessee_County_Vaccine_Race.to_csv('FinalResult/Tennessee.csv')
print(Tennessee_County_Vaccine_Race.shape)
Tennessee_County_Vaccine_Race.head() # 95 counties in Tennessee

(96, 9)


,County,ASIAN,BLACK OR AFRICAN AMERICAN,OTHER/MULTIRACIAL,RACE-UNKNOWN,WHITE,HISPANIC OR LATINO,NOT HISPANIC OR LATINO,ETHN-UNKNOWN
0,ANDERSON,281.0,543.0,4614.0,2610.0,19904.0,776.0,23338.0,3838.0
1,BEDFORD,100.0,642.0,1332.0,964.0,7774.0,576.0,8840.0,1396.0
2,BENTON,26.0,185.0,541.0,113.0,4178.0,73.0,4738.0,232.0
3,BLEDSOE,NaN,33.0,343.0,284.0,2380.0,45.0,2537.0,461.0
4,BLOUNT,273.0,659.0,5508.0,3353.0,35170.0,727.0,38953.0,5283.0


In [20]:

# Tennessee_County_Vaccine_Race.to_csv('FinalResult/Tennessee.csv')
print(Tennessee_County_Vaccine_Race.shape)
Tennessee_County_Vaccine_Race.head() # 95 counties in Tennessee

(96, 9)


,County,ASIAN,BLACK OR AFRICAN AMERICAN,OTHER/MULTIRACIAL,RACE-UNKNOWN,WHITE,HISPANIC OR LATINO,NOT HISPANIC OR LATINO,ETHN-UNKNOWN
0,ANDERSON,281.0,543.0,4614.0,2610.0,19904.0,776.0,23338.0,3838.0
1,BEDFORD,100.0,642.0,1332.0,964.0,7774.0,576.0,8840.0,1396.0
2,BENTON,26.0,185.0,541.0,113.0,4178.0,73.0,4738.0,232.0
3,BLEDSOE,NaN,33.0,343.0,284.0,2380.0,45.0,2537.0,461.0
4,BLOUNT,273.0,659.0,5508.0,3353.0,35170.0,727.0,38953.0,5283.0


In [21]:
# print(len(L))

import numpy as np



state = 'Tennessee' ## 

new_L = []


for idx, row in Tennessee_County_Vaccine_Race.iterrows(): ## 
    d = row.to_dict()
    
    new_d = {}
    
    
    new_d['State'] = state
    new_d['County'] = ' '.join([i.lower().capitalize() for i in d['County'].split(' ')])
    RenameCounties = {
        'Dekalb': 'DeKalb',
        'Mcminn': 'McMinn', 
        'Mcnairy': 'McNairy',
        # 'Mccormick': 'McCormick'
    }
    if new_d['County'] in RenameCounties:
        new_d['County'] = RenameCounties[new_d['County']]

    new_d['Geographic Area Name'] = new_d['County'] + ', ' + new_d['State']
        
        
    new_d['Numb_White'] = d['WHITE']
    
    new_d['Numb_Black'] = d['BLACK OR AFRICAN AMERICAN']
    
    
    new_d['Numb_Asian'] = d['ASIAN']
    new_d['Rate_Asian'] = np.nan
    
    
    new_d['Date'] = np.nan
    
    new_L.append(new_d)
    
    
Data = pd.DataFrame(new_L)
not_included = list(set(Data['Geographic Area Name'].unique()) - set(US_COUNTY_POPULATION['Geographic Area Name'].unique()))
if len(not_included) > 0:
    print('Not included Counties', not_included)


T = pd.merge(Data, US_COUNTY_POPULATION[US_COUNTY_POPULATION.columns[3:]], on = 'Geographic Area Name')


for Race in ['White', 'Black', 'Asian']:
    Rate = T['Numb_' + Race] / T['Total_' + Race]
    Rate = Rate.replace([-np.inf, np.inf], np.nan)
    T['Rate_' + Race] = Rate

#     Rate = T['Numb_' + Race] / T['Total_' + Race + 'C']
#     Rate = Rate.replace([-np.inf, np.inf], np.nan)
#     T['Rate_' + Race + 'C'] = Rate


T = T[list(T.columns[:3]) + list(T.columns[3:].sort_values())]


print(T.shape)
T.head()

Not included Counties ['Out Of State, Tennessee']
(95, 19)


,State,County,Geographic Area Name,Date,Numb_Asian,Numb_Black,Numb_White,Rate_Asian,Rate_Black,Rate_White,Total_AIndA,Total_AIndAC,Total_Asian,Total_AsianC,Total_Black,Total_BlackC,Total_White,Total_WhiteC,Total_Whole
0,Tennessee,Anderson,"Anderson, Tennessee",NaN,281.0,543.0,19904.0,0.239761,0.172162,0.282422,371,911,1172,1480,3154,4161,70476,72149,76978
1,Tennessee,Bedford,"Bedford, Tennessee",NaN,100.0,642.0,7774.0,0.192308,0.152894,0.179733,580,843,520,669,4199,4922,43253,44280,49713
2,Tennessee,Benton,"Benton, Tennessee",NaN,26.0,185.0,4178.0,0.265306,0.441527,0.273304,91,220,98,136,419,536,15287,15532,16160
3,Tennessee,Bledsoe,"Bledsoe, Tennessee",NaN,NaN,33.0,2380.0,NaN,0.028448,0.175685,96,188,45,71,1160,1261,13547,13750,15064
4,Tennessee,Blount,"Blount, Tennessee",NaN,273.0,659.0,35170.0,0.210000,0.163930,0.281897,564,1487,1300,1754,4020,5075,124762,126972,133088


In [22]:
T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']
# T['Rate_ParityC'] = T['Rate_WhiteC'] - T['Rate_BlackC']
STATE_VACCINE[state] = T
print([i for i in STATE_VACCINE])
diagnose_Rate(T)

['California', 'Tennessee']


,name,count,mean,std,min,25%,50%,75%,max
0,Rate_Black,90.0,0.194197,0.102404,0.028448,0.141727,0.175535,0.218637,0.741855
1,Rate_White,95.0,0.244397,0.051267,0.134284,0.209779,0.232962,0.274155,0.416109
2,Rate_Parity,90.0,0.052561,0.099532,-0.508893,0.026664,0.063320,0.110106,0.184551


In [23]:
# T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']
# # T['Rate_ParityC'] = T['Rate_WhiteC'] - T['Rate_BlackC']
# STATE_VACCINE[state] = T
# print([i for i in STATE_VACCINE])
# diagnose_Rate(T)

## [!] B. North Carolina* (Done)

Download the Data First

https://covid19.ncdhhs.gov/dashboard/data-behind-dashboards

In [24]:
################################################
# path = 'TMP/NorthCarolina/NC_Mar26_Doses by County.xlsx'
# path = 'TMP/Template/NorthCarolina/NC_April2_Vaccination_Race.xlsx'

################################################

'TMP/April09/North Carolina/Vaccination_Race.xlsx'


'TMP/April10/NorthCarolina/Vaccination_Race.xlsx'


'TMP/April12/NorthCarolina/Vaccination_Race.xlsx'


'TMP/April14/North Carolina/Vaccination_Race14.xls'


'TMP/April16/NorthCarolina/Vaccination_Race-NorthCaro-0416.xlsx'

'TMP/April16/NorthCarolina/Vaccination_Race-NorthCaro-0416.xlsx'

In [25]:




Date = Today_Date
import os

while True:
    Path = input('Enter North Carolina Manually collected Data:  ')
    if os.path.isfile(Path):
        print('File Exists!')
        x = input('Enter "Yes" if the file contains the information of the date you wanted:  ')
        if x =='Yes':
            break
    else:
        print('Wrong file path...')
        
        
print('\nThe Path we have:')
print(Path)


Data = pd.read_excel(Path)
Data.columns



Enter North Carolina Manually collected Data:  TMP/April16/NorthCarolina/Vaccination_Race-NorthCaro-0416.xlsx
File Exists!
Enter "Yes" if the file contains the information of the date you wanted:  Yes

The Path we have:
TMP/April16/NorthCarolina/Vaccination_Race-NorthCaro-0416.xlsx


Index(['Index', 'County ', 'Week of', 'American Indian or Alaskan Native',
       'Asian or Pacific Islander', 'Black or African American', 'White',
       'Other', 'Suppressed', 'Missing or Undisclosed'],
      dtype='object')

In [26]:

columns = ['County ', 'Week of', 'American Indian or Alaskan Native',
       'Asian or Pacific Islander', 'Black or African American',
       'Missing or Undisclosed', 'Other', 'Suppressed', 'White']


NorthCarolina = Data[columns]
Groups = NorthCarolina.groupby('County ')

CL = []


for County, df in Groups:
    D = {}
    D['County'] = County
    d = df.sum().to_dict() # Here is a sum
    D['Black or African American'] = d['Black or African American']
    D['American Indian or Alaskan Native'] = d['American Indian or Alaskan Native']
    D['White'] = d['White']
    D['Asian or Pacific Islander'] = d['Asian or Pacific Islander']
    
    CL.append(D)
    
    
NC = pd.DataFrame(CL)
NC = NC.replace(0.0, np.nan) ### Changes here
NC 

,County,Black or African American,American Indian or Alaskan Native,White,Asian or Pacific Islander
0,Alamance,9950.0,189.0,36823.0,1146.0
1,Alexander,NaN,NaN,9005.0,NaN
2,Alleghany,NaN,NaN,2827.0,NaN
3,Anson,2623.0,NaN,NaN,NaN
4,Ashe,NaN,NaN,7687.0,NaN
...,...,...,...,...,...
97,Wayne,7794.0,115.0,16085.0,482.0
98,Wilkes,NaN,NaN,14617.0,NaN
99,Wilson,7071.0,48.0,12148.0,235.0
100,Yadkin,NaN,NaN,9297.0,NaN


In [27]:
# print(len(L))

import numpy as np



state = 'North Carolina' ## 

new_L = []


for idx, row in NC.iterrows(): ## 
    d = row.to_dict()
    # print(d)
    new_d = {}
    
    if type(d['County']) != str:
        print(d)
        continue 
        
    new_d['State'] = state
    new_d['County'] = ' '.join([i.lower().capitalize() for i in d['County'].split(' ')])
    RenameCounties = {
        # 'Mckean': 'McKean'
        'Mcdowell': 'McDowell'
    }
    if new_d['County'] in RenameCounties:
        new_d['County'] = RenameCounties[new_d['County']]

    new_d['Geographic Area Name'] = new_d['County'] + ', ' + new_d['State']
    
    
    new_d['Numb_White'] = d['White']
    new_d['Rate_White'] = np.nan
    
    new_d['Numb_Black'] = d['Black or African American']
    new_d['Rate_Black'] = np.nan
    
    new_d['Numb_Asian'] = d['Asian or Pacific Islander']
    new_d['Rate_Asian'] = np.nan
    
    new_d['Numb_AIndA'] = d['American Indian or Alaskan Native']
    new_d['Rate_AIndA'] = np.nan
    
    
    new_d['Date'] = np.nan
    
    new_L.append(new_d)
    
    
Data = pd.DataFrame(new_L)
not_included = list(set(Data['Geographic Area Name'].unique()) - set(US_COUNTY_POPULATION['Geographic Area Name'].unique()))
if len(not_included) > 0:
    print('Not included Counties', not_included)


T = pd.merge(Data, US_COUNTY_POPULATION[US_COUNTY_POPULATION.columns[3:]], on = 'Geographic Area Name')


for Race in ['White', 'Black', 'Asian']:
    Rate = T['Numb_' + Race] / T['Total_' + Race]
    Rate = Rate.replace([-np.inf, np.inf], np.nan)
    T['Rate_' + Race] = Rate

#     Rate = T['Numb_' + Race] / T['Total_' + Race + 'C']
#     Rate = Rate.replace([-np.inf, np.inf], np.nan)
#     T['Rate_' + Race + 'C'] = Rate


T = T[list(T.columns[:3]) + list(T.columns[3:].sort_values())]

print(T.shape)
T.head()

Not included Counties ['Missing, North Carolina', 'Out Of State, North Carolina']
(100, 21)


,State,County,Geographic Area Name,Date,Numb_AIndA,Numb_Asian,Numb_Black,Numb_White,Rate_AIndA,Rate_Asian,...,Rate_White,Total_AIndA,Total_AIndAC,Total_Asian,Total_AsianC,Total_Black,Total_BlackC,Total_White,Total_WhiteC,Total_Whole
0,North Carolina,Alamance,"Alamance, North Carolina",NaN,189.0,1146.0,9950.0,36823.0,NaN,0.387424,...,0.295302,2466,3610,2958,3794,35369,37943,124696,128147,169509
1,North Carolina,Alexander,"Alexander, North Carolina",NaN,NaN,NaN,NaN,9005.0,NaN,NaN,...,0.264487,199,376,385,470,2214,2630,34047,34625,37497
2,North Carolina,Alleghany,"Alleghany, North Carolina",NaN,NaN,NaN,NaN,2827.0,NaN,NaN,...,0.265945,59,144,89,107,197,269,10630,10766,11137
3,North Carolina,Anson,"Anson, North Carolina",NaN,NaN,NaN,2623.0,NaN,NaN,NaN,...,NaN,227,368,360,419,11846,12094,11588,11960,24446
4,North Carolina,Ashe,"Ashe, North Carolina",NaN,NaN,NaN,NaN,7687.0,NaN,NaN,...,0.291705,114,269,138,176,262,416,26352,26643,27203


In [28]:
T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']
# T['Rate_ParityC'] = T['Rate_WhiteC'] - T['Rate_BlackC']
STATE_VACCINE[state] = T
print([i for i in STATE_VACCINE])
diagnose_Rate(T)

['California', 'Tennessee', 'North Carolina']


,name,count,mean,std,min,25%,50%,75%,max
0,Rate_Black,52.0,0.231583,0.045489,0.141241,0.206190,0.230632,0.264359,0.334077
1,Rate_White,91.0,0.284953,0.063573,0.126731,0.241847,0.271267,0.323041,0.465488
2,Rate_Parity,43.0,0.045394,0.044575,-0.036891,0.015741,0.033184,0.077415,0.157794


# IV. OCR States

## [!] C. West Virgina* (Done)


Copy Response from Browsers

https://dhhr.wv.gov/COVID-19/Pages/default.aspx

url = 'https://wabi-us-gov-virginia-api.analysis.usgovcloudapi.net/public/reports/querydata?synchronous=true'




In [29]:
'TMP/April09/West Virginia/WestVirginiaApril09.json'

'TMP/April10/WestVirginia/WVApril10.json'


'TMP/April12/WestVirginia/WVApril012.json'



'TMP/April14/West Virginia/WestVirginiaApril14.json'


'TMP/April16/WestVirginia/West Virginia - 0416.json'

'TMP/April16/WestVirginia/West Virginia - 0416.json'

In [30]:


Date = Today_Date
import os

while True:
    Path = input('Enter West Virginia Manually collected Data:  ')
    if os.path.isfile(Path):
        print('File Exists!')
        x = input('Enter "Yes" if the file contains the information of the date you wanted:  ')
        if x =='Yes':
            break
    else:
        print('Wrong file path...')
        
        
print('\nThe Path we have:')
print(Path)


Enter West Virginia Manually collected Data:  TMP/April16/WestVirginia/West Virginia - 0416.json
File Exists!
Enter "Yes" if the file contains the information of the date you wanted:  Yes

The Path we have:
TMP/April16/WestVirginia/West Virginia - 0416.json


In [31]:

import requests
import json


date = Today_Date


with open(Path, 'r') as f:
    response_data = json.load(f)

def get_WestVirginia_Raw(date, response_data):
    

    # data = json.loads(response_data)
    data = response_data

    tmp = data['results'][0]['result']['data']['dsr']['DS'][0]
    # [i for i in tmp]

    tmp2 = tmp['PH'][1]['DM1']
    # len(tmp2)
    # tmp2


    L = []
    for county_info in tmp2:
        d= {}
        d['County'] = county_info['G0']
        d['Date' ] =  date
        try:
            info = [race['C'] for race in county_info['X'][:-1]]

            d['Numb_Black'] = info[0][0]
            d['Numb_White'] = info[-1][0]
            L.append(d)
        except:
            print(d['County'])
            print(county_info['X'])


    WestVirginia = pd.DataFrame(L)
    return WestVirginia
    
    
WestVirginia = get_WestVirginia_Raw(date, response_data)
# WestVirginia.to_csv('FinalResult/Post_WestVirginia.csv')
print(WestVirginia.shape)
WestVirginia.head() # 55 counties


(55, 4)


,County,Date,Numb_Black,Numb_White
0,Barbour,2021-04-17,25,4003
1,Berkeley,2021-04-17,1830,25047
2,Boone,2021-04-17,37,5848
3,Braxton,2021-04-17,16,4510
4,Brooke,2021-04-17,45,6857


In [32]:
# print(len(L))

import numpy as np

state = 'West Virginia' ## 

new_L = []

for idx, row in WestVirginia.iterrows(): ## 
    d = row.to_dict()
    # print(d)
    new_d = {}
    
        
    new_d['State'] = state
    new_d['County'] = ' '.join([i.lower().capitalize() for i in d['County'].split(' ')])
    RenameCounties = {
        # 'Mckean': 'McKean'
     'Mcdowell': 'McDowell'
    }
    if new_d['County'] in RenameCounties:
        new_d['County'] = RenameCounties[new_d['County']]

    new_d['Geographic Area Name'] = new_d['County'] + ', ' + new_d['State']

    new_d['Numb_White'] = d['Numb_White']
    # new_d['Rate_White'] = np.nan
    
    new_d['Numb_Black'] = d['Numb_Black']
    
    
    new_d['Date'] = d['Date']
    
    new_L.append(new_d)
    
    
Data = pd.DataFrame(new_L)


not_included = list(set(Data['Geographic Area Name'].unique()) - set(US_COUNTY_POPULATION['Geographic Area Name'].unique()))
if len(not_included) > 0:
    print('Not included Counties', not_included)


T = pd.merge(Data, US_COUNTY_POPULATION[US_COUNTY_POPULATION.columns[3:]], on = 'Geographic Area Name')


for Race in ['White', 'Black']:
    Rate = T['Numb_' + Race] / T['Total_' + Race]
    Rate = Rate.replace([-np.inf, np.inf], np.nan)
    T['Rate_' + Race] = Rate

    Rate = T['Numb_' + Race] / T['Total_' + Race + 'C']
    Rate = Rate.replace([-np.inf, np.inf], np.nan)
    T['Rate_' + Race + 'C'] = Rate


T = T[list(T.columns[:3]) + list(T.columns[3:].sort_values())]

print(T.shape)
T.head()

(55, 19)


,State,County,Geographic Area Name,Date,Numb_Black,Numb_White,Rate_Black,Rate_BlackC,Rate_White,Rate_WhiteC,Total_AIndA,Total_AIndAC,Total_Asian,Total_AsianC,Total_Black,Total_BlackC,Total_White,Total_WhiteC,Total_Whole
0,West Virginia,Barbour,"Barbour, West Virginia",2021-04-17,25,4003,0.150602,0.089286,0.252380,0.248587,106,225,50,91,166,280,15861,16103,16441
1,West Virginia,Berkeley,"Berkeley, West Virginia",2021-04-17,1830,25047,0.194681,0.154901,0.239712,0.231998,300,1074,1328,2011,9400,11814,104488,107962,119171
2,West Virginia,Boone,"Boone, West Virginia",2021-04-17,37,5848,0.232704,0.142308,0.277617,0.275485,33,95,27,41,159,260,21065,21228,21457
3,West Virginia,Braxton,"Braxton, West Virginia",2021-04-17,16,4510,0.161616,0.089888,0.332008,0.327976,61,135,33,62,99,178,13584,13751,13957
4,West Virginia,Brooke,"Brooke, West Virginia",2021-04-17,45,6857,0.138462,0.087209,0.324438,0.319763,41,148,110,157,325,516,21135,21444,21939


In [33]:
# West Virginia	Barbour	Barbour, West Virginia	2021-03-26	10	2839	0.060241	0.035714	0.178992	0.176303	106	225	50

T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']
# T['Rate_ParityC'] = T['Rate_WhiteC'] - T['Rate_BlackC']
STATE_VACCINE[state] = T
print([i for i in STATE_VACCINE])
diagnose_Rate(T)

['California', 'Tennessee', 'North Carolina', 'West Virginia']


,name,count,mean,std,min,25%,50%,75%,max
0,Rate_Black,55.0,0.185551,0.097364,0.016548,0.108497,0.161616,0.268325,0.377011
1,Rate_White,55.0,0.334019,0.054645,0.228942,0.304338,0.328622,0.373696,0.477988
2,Rate_Parity,55.0,0.148468,0.088079,-0.020616,0.097364,0.140913,0.218633,0.325899


# VI. Manual States

## [!] E. Maine* (Done)

In [37]:
'TMP/April09/Maine/Maine April 8.xls'

'TMP/April10/Maine/Maine-FirstDose.csv'


'TMP/April12/Maine/Maine-0412.csv'


'TMP/April14/Maine/Maine April 14.xls'


'TMP/April16/Maine/Maine-0416.csv'

'TMP/April16/Maine/Maine-0416.csv'

In [38]:
import os


Date = Today_Date


while True:
    Path = input('Enter Maine Manually collected Data:  ')
    if os.path.isfile(Path):
        print('File Exists!')
        x = input('Enter "Yes" if the file contains the information of the date you wanted:  ')
        if x =='Yes':
            break
    else:
        print('Wrong file path...')
        
        
print('\nThe Path we have:')
print(Path)


Enter Maine Manually collected Data:  TMP/April16/Maine/Maine-0416.csv
File Exists!
Enter "Yes" if the file contains the information of the date you wanted:  Yes

The Path we have:
TMP/April16/Maine/Maine-0416.csv


In [39]:



try:
    Data = pd.read_csv(Path)
except:
    Data = pd.read_excel(Path)
    
Data = Data[Data.columns[1:]]
Maine = Data
# Maine.to_csv('FinalResult/Maine.csv')
Maine = Maine.rename(columns = {'Black or African American': 'Black'})
Maine

,State full,County,Black,White
0,Maine,ANDROSCOGGIN,591,28510
1,Maine,AROOSTOOK,92,19513
2,Maine,CUMBERLAND,2051,110701
3,Maine,FRANKLIN,16,9006
4,Maine,HANCOCK,99,20953
5,Maine,KENNEBEC,238,36675
6,Maine,KNOX,57,15012
7,Maine,LINCOLN,39,14633
8,Maine,OXFORD,73,17426
9,Maine,PENOBSCOT,373,48357


In [40]:
# print(len(L))

import numpy as np
state = 'Maine' ## 

new_L = []
for idx, row in Maine.iterrows(): ## 
    d = row.to_dict()
    
    new_d = {}
    
    
    new_d['State'] = state
    new_d['County'] = ' '.join([i.lower().capitalize() for i in d['County'].split(' ')])
    new_d['Geographic Area Name'] = new_d['County'] + ', ' + new_d['State']
    
    new_d['Numb_White'] = d['White'] if d['White']!= '-' else np.nan
    # new_d['Rate_White'] = np.nan
    
    new_d['Numb_Black'] = d['Black'] if d['Black']!= '-' else np.nan
    # new_d['Rate_Black'] = np.nan

    
    # new_d['Date'] = np.nan
    
    new_L.append(new_d)
    
    
Data = pd.DataFrame(new_L)
not_included = list(set(Data['Geographic Area Name'].unique()) - set(US_COUNTY_POPULATION['Geographic Area Name'].unique()))
if len(not_included) > 0:
    print('Not included Counties', not_included)


T = pd.merge(Data, US_COUNTY_POPULATION[US_COUNTY_POPULATION.columns[3:]], on = 'Geographic Area Name')


for Race in ['White', 'Black']:
    Rate = T['Numb_' + Race] / T['Total_' + Race]
    Rate = Rate.replace([-np.inf, np.inf], np.nan)
    T['Rate_' + Race] = Rate


T = T[list(T.columns[:3]) + list(T.columns[3:].sort_values())]
print(T.shape)
T.head()

(16, 16)


,State,County,Geographic Area Name,Numb_Black,Numb_White,Rate_Black,Rate_White,Total_AIndA,Total_AIndAC,Total_Asian,Total_AsianC,Total_Black,Total_BlackC,Total_White,Total_WhiteC,Total_Whole
0,Maine,Androscoggin,"Androscoggin, Maine",591,28510,0.121305,0.287072,457,1251,1014,1685,4872,6145,99313,101811,108277
1,Maine,Aroostook,"Aroostook, Maine",92,19513,0.115578,0.306967,1293,1922,343,516,796,1063,63567,64576,67055
2,Maine,Cumberland,"Cumberland, Maine",2051,110701,0.217014,0.407899,1037,2945,7057,9344,9451,11622,271393,277223,295003
3,Maine,Franklin,"Franklin, Maine",16,9006,0.095808,0.308224,146,440,160,259,167,299,29219,29696,30199
4,Maine,Hancock,"Hancock, Maine",99,20953,0.174603,0.397824,261,664,684,878,567,805,52669,53436,54987


In [41]:
T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']
STATE_VACCINE[state] = T
print([i for i in STATE_VACCINE])
diagnose_Rate(T)

['California', 'Tennessee', 'North Carolina', 'West Virginia', 'Maine']


,name,count,mean,std,min,25%,50%,75%,max
0,Rate_Black,16.0,0.167753,0.051237,0.076923,0.125528,0.175537,0.197981,0.249833
1,Rate_White,16.0,0.335583,0.053566,0.242098,0.307910,0.332530,0.368169,0.436558
2,Rate_Parity,16.0,0.167830,0.058034,0.072982,0.127351,0.164266,0.214432,0.261588


## [!]  E. New Jersey* (Done)

In [42]:

'TMP/April09/New Jersey/New Jersey 4 9.xls'

'TMP/April10/NewJersey/New Jersey-First Dose.csv'


'TMP/April12/NewJersey/New Jersey-0412.csv'


'TMP/April14/New Jersey/New Jersey 4_14.xls'


'TMP/April16/NewJersey/New Jersey-0416.csv'

'TMP/April16/NewJersey/New Jersey-0416.csv'

In [43]:
import os

Date = Today_Date


while True:
    Path = input('Enter **New Jersey** Manually collected Data:  ')
    if os.path.isfile(Path):
        print('File Exists!')
        x = input('Enter "Yes" if the file contains the information of the date you wanted:  ')
        if x =='Yes':
            break
    else:
        print('Wrong file path...')
        
        
print('\nThe Path we have:')
print(Path)


Enter **New Jersey** Manually collected Data:  TMP/April16/NewJersey/New Jersey-0416.csv
File Exists!
Enter "Yes" if the file contains the information of the date you wanted:  Yes

The Path we have:
TMP/April16/NewJersey/New Jersey-0416.csv


In [44]:
try:
    Data = pd.read_csv(Path)
except:
    Data = pd.read_excel(Path)
Data = Data[Data.columns[1:]]

NewJersey = Data

# NewJersey.to_csv('FinalResult/NewJersey.csv')

NewJersey.head()

,State full,County,People with at least one vaccine dose,Black,White
0,New Jersey,ATLANTIC,113307,7,60
1,New Jersey,BERGEN,402504,3,53
2,New Jersey,BURLINGTON,188329,5,69
3,New Jersey,CAMDEN,210610,8,63
4,New Jersey,CAPE MAY,44345,2,79


In [46]:
# NewJersey[['Black', 'White'] ].sum(axis=1)

In [47]:
# print(len(L))

import numpy as np

state = 'New Jersey' ## 

new_L = []


for idx, row in NewJersey.iterrows(): ## 
    d = row.to_dict()
    
    new_d = {}
    
    
    new_d['State'] = state
    new_d['County'] = ' '.join([i.lower().capitalize() for i in d['County'].split(' ')])
    new_d['Geographic Area Name'] = new_d['County'] + ', ' + new_d['State']
    
    new_d['Numb_White'] = d['People with at least one vaccine dose'] *  d['White'] / 100
    
    new_d['Numb_Black'] = d['People with at least one vaccine dose'] *  d['Black'] / 100
    
    
    # new_d['Numb_Asian'] = d['First Dose Total'] *  d['Asian %'] / 100
    
    
    # new_d['Numb_Hispa'] = d['First Dose Total'] * d['Hispanic/Latinx %'] / 100
    
    new_d['Date'] = np.nan
    
    new_L.append(new_d)
    
    
Data = pd.DataFrame(new_L)
not_included = list(set(Data['Geographic Area Name'].unique()) - set(US_COUNTY_POPULATION['Geographic Area Name'].unique()))
if len(not_included) > 0:
    print('Not included Counties', not_included)


T = pd.merge(Data, US_COUNTY_POPULATION[US_COUNTY_POPULATION.columns[3:]], on = 'Geographic Area Name')


for Race in ['White', 'Black']:
    Rate = T['Numb_' + Race] / T['Total_' + Race]
    Rate = Rate.replace([-np.inf, np.inf], np.nan)
    T['Rate_' + Race] = Rate

    Rate = T['Numb_' + Race] / T['Total_' + Race + 'C']
    Rate = Rate.replace([-np.inf, np.inf], np.nan)
    T['Rate_' + Race + 'C'] = Rate


T = T[list(T.columns[:3]) + list(T.columns[3:].sort_values())]


print(T.shape)
T.head()

(21, 19)


,State,County,Geographic Area Name,Date,Numb_Black,Numb_White,Rate_Black,Rate_BlackC,Rate_White,Rate_WhiteC,Total_AIndA,Total_AIndAC,Total_Asian,Total_AsianC,Total_Black,Total_BlackC,Total_White,Total_WhiteC,Total_Whole
0,New Jersey,Atlantic,"Atlantic, New Jersey",NaN,7931.49,67984.20,0.175549,0.160045,0.361651,0.350315,1782,3466,21282,23764,45181,49558,187983,194066,263670
1,New Jersey,Bergen,"Bergen, New Jersey",NaN,12075.12,213327.12,0.176199,0.153465,0.314039,0.305825,4508,8451,158020,168160,68531,78683,679302,697546,932202
2,New Jersey,Burlington,"Burlington, New Jersey",NaN,9416.45,129947.01,0.115518,0.104472,0.400470,0.387207,1520,4758,24118,28721,81515,90134,324486,335601,445349
3,New Jersey,Camden,"Camden, New Jersey",NaN,16848.80,132684.30,0.153811,0.142250,0.379600,0.367907,3304,6565,30391,34188,109542,118445,349537,360646,506471
4,New Jersey,Cape May,"Cape May, New Jersey",NaN,886.90,35032.55,0.199169,0.160845,0.414847,0.407019,336,723,943,1366,4453,5514,84447,86071,92039


In [48]:
T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']
# T['Rate_ParityC'] = T['Rate_WhiteC'] - T['Rate_BlackC']
STATE_VACCINE[state] = T
print([i for i in STATE_VACCINE])
diagnose_Rate(T)

['California', 'Tennessee', 'North Carolina', 'West Virginia', 'Maine', 'New Jersey']


,name,count,mean,std,min,25%,50%,75%,max
0,Rate_Black,21.0,0.162441,0.040041,0.093982,0.151806,0.164429,0.175549,0.263273
1,Rate_White,21.0,0.325715,0.062730,0.175946,0.283733,0.341774,0.361651,0.414847
2,Rate_Parity,21.0,0.163274,0.050845,0.051691,0.124928,0.171787,0.186102,0.284952


# All in One

## Read Previous Vaccine Data

In [49]:
# import os
# path = ''

# print('Input Historical STATE_VACCINE')
# print('For example: RaceVaccineBackup/STATE_VACCINE_2021-03-23.p')

# L = []
# while True:
    
#     path = input()
    
#     if os.path.isfile(path):
#         print('Path Exists!')
#         L.append(path)
#     elif path != '':
#         print('Wrong Path')
#     else:
#         break

# print('End')

In [50]:
# import pickle

# Historical_Path = {i.split('_')[-1].replace('.p', ''): i for i in L }

# Historical_Data = {}
# for date, path in Historical_Path.items():
#     with open(path, 'rb' ) as f:
#         Historical_Data[date] = pickle.load(f)
    
    
# [i for i in Historical_Data]

## Show Parity for Each State

In [52]:
# ######################################

# print(Today_Date)
# ######################################

# for state in STATE_VACCINE:
    
#     print(state)
#     cols = ['name',  'date', 'count', 'mean', 'min','50%', 'max']

    
#     for date, Old_STATE_VACCINE in Historical_Data.items():
#         x = diagnose_Rate(Old_STATE_VACCINE[state], state = state, date = date)
#         print(x[cols])
    
#     x = diagnose_Rate(STATE_VACCINE[state], state = state, date = Today_Date)
    
#     print(x[cols])
#     print('\n')
    
    
# # pd.concat(L).to_csv('compare.csv')

##  Show Some Specials Cases

In [53]:
# # state = 'Texas'
# DF = pd.concat([v for k,v in STATE_VACCINE.items()]) 

# data_cols = ['State', 'County', 'Geographic Area Name',
#              'Total_Black', 'Total_White',
#              'Numb_Black', 'Numb_White','Rate_Black','Rate_White', 'Total_Whole', 'Rate_Parity']

# DF[DF['Rate_Black'] > 0.3][data_cols]

## Save Data

In [54]:

import pickle 

path = 'Workplace/STATE_VACCINE_' + Today_Date + '_Manual.p'

while True:
    print('The data will be saved here, do you agree?' )
    print(path)
    print('If yes, please enter: Yes')
    x = input()
    if x =='Yes':
        break


with open(path, 'wb') as f:
    pickle.dump(STATE_VACCINE, f)
    
    
print('Successfully Saved!')

The data will be saved here, do you agree?
Workplace/STATE_VACCINE_2021-04-17_Manual.p
If yes, please enter: Yes
Yes
Successfully Saved!


# Merge Data

In [57]:
def merge_data(date):
    D = {}
    for suffix in ['_Manual.p', '_Tableau.p', '_Autonomy.p']:
        path = 'Workplace/STATE_VACCINE_' + date + suffix
        print(path)
        with open(path, 'rb') as f:
            s = pickle.load(f)
            
        for k, v in s.items():
            D[k] = v
            
    return D
    
    
Today_Date = '2021-04-16'
date = Today_Date
path = 'STATE_VACCINE_' + date + '.p'
D = merge_data(date)


Workplace/STATE_VACCINE_2021-04-16_Manual.p
Workplace/STATE_VACCINE_2021-04-16_Tableau.p
Workplace/STATE_VACCINE_2021-04-16_Autonomy.p


In [58]:
STATE_VACCINE = D

with open(path, 'wb') as f:
    pickle.dump(STATE_VACCINE, f)

print('save data to:', path)


save data to: STATE_VACCINE_2021-04-16.p
